### ** Read fact_live_times and clean dataset**

In [5]:
import pandas as pd
import numpy as np

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 7, Finished, Available, Finished)

In [6]:
# Load data into pandas DataFrame 
fact_live_times = spark.read.table("fact_live_times").toPandas()

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 8, Finished, Available, Finished)

In [7]:
fact_live_times.describe()

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 9, Finished, Available, Finished)

,PopularTimesLivePercent,DateTimeCollected,Datetime,Hour,DateKey,IsWeekend,IsHoliday,DayOfWeek,HourKey,WeatherKey,Temp,IsRaining,IsSnowing,ParkKey,AmenityKey,EventKey,HasEvent,FactLiveKey
count,330169.000000,386533,386533,386533.000000,3.865330e+05,386533.000000,386533.000000,386533.000000,386533.000000,3.865330e+05,384935.000000,384935.000000,384935.000000,3.865330e+05,3.615110e+05,3.865330e+05,386533.000000,3.865330e+05
mean,35.852170,2024-11-15 06:32:29.175745,2024-11-15 06:13:06.381498,14.004331,2.024443e+07,0.286154,0.028569,4.001092,14.004331,2.141302e+09,11.681559,0.219551,0.006056,1.621304e+09,2.121350e+09,9.814628e+06,0.004476,2.147581e+09
min,-100.000000,2024-04-30 06:12:56.020000,2024-04-30 06:00:00,6.000000,2.024043e+07,0.000000,0.000000,1.000000,6.000000,-1.000000e+00,-4.700000,0.000000,0.000000,-1.000000e+00,3.311490e+07,-1.000000e+00,0.000000,4.964000e+03
25%,9.000000,2024-08-15 20:15:16.932000,2024-08-15 20:00:00,10.000000,2.024082e+07,0.000000,0.000000,2.000000,10.000000,1.060806e+09,7.200000,0.000000,0.000000,4.554244e+08,1.152139e+09,-1.000000e+00,0.000000,1.073777e+09
50%,29.000000,2024-11-16 20:16:30.199000,2024-11-16 20:00:00,14.000000,2.024112e+07,0.000000,0.000000,4.000000,14.000000,2.143269e+09,10.900000,0.000000,0.000000,1.370607e+09,2.095798e+09,-1.000000e+00,0.000000,2.147935e+09
75%,56.000000,2025-02-15 18:26:19.526000,2025-02-15 18:00:00,18.000000,2.025022e+07,1.000000,0.000000,6.000000,18.000000,3.229024e+09,16.000000,0.000000,0.000000,2.684997e+09,3.289091e+09,-1.000000e+00,0.000000,3.221300e+09
max,100.000000,2025-05-15 22:26:39.342000,2025-05-15 22:00:00,22.000000,2.025052e+07,1.000000,1.000000,7.000000,22.000000,4.293553e+09,30.200000,1.000000,1.000000,4.157014e+09,4.240448e+09,4.285014e+09,1.000000,4.294932e+09
std,30.828232,NaN,NaN,4.898386,4.561835e+03,0.451963,0.166593,2.000044,4.898386,1.247344e+09,6.130415,0.413943,0.077582,1.302749e+09,1.264130e+09,1.678456e+08,0.066751,1.239681e+09


In [8]:
# Check Duplicate
duplicate_count = fact_live_times.duplicated().sum()
print(f"Number of duplicated rows: {duplicate_count}")

# Check Missing Value
missing_summary = fact_live_times.isna().mean().mul(100).round(2).to_frame('MissingPercent')
missing_summary['MissingCount'] = fact_live_times.isna().sum()
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].sort_values('MissingCount', ascending=False)

print(missing_summary)

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 10, Finished, Available, Finished)

Number of duplicated rows: 0
                         MissingPercent  MissingCount
PopularTimesLivePercent           14.58         56364
AmenityKey                         6.47         25022
ParkName                           6.42         24814
Temp                               0.41          1598
IsRaining                          0.41          1598
IsSnowing                          0.41          1598


In [9]:
# Required columns 
required_cols = ['DateTimeCollected','ParkName', 'Temp', 'Hour', 'IsWeekend', 'IsHoliday', 'IsRaining', 'IsSnowing', 'HasEvent', 'DayOfWeek']

# Report
print(f"Duplicates: {fact_live_times.duplicated(subset=required_cols).sum()}")
print("Missing values:\n", fact_live_times[required_cols].isna().sum().loc[lambda x: x > 0])

# Clean duplicate and missing value except target 
fact_live_times = fact_live_times.drop_duplicates(subset=required_cols).dropna(subset=required_cols).copy()

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 11, Finished, Available, Finished)

Duplicates: 18463
Missing values:
 ParkName     24814
Temp          1598
IsRaining     1598
IsSnowing     1598
dtype: int64


#### **Pre-checking prior imputation of target variable 'PopularTimesLivePercent'**

In [10]:
# Check Missing Value
missing_summary = fact_live_times.isna().mean().mul(100).round(2).to_frame('MissingPercent')
missing_summary['MissingCount'] = fact_live_times.isna().sum()
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].sort_values('MissingCount', ascending=False)

print(missing_summary)

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 12, Finished, Available, Finished)

                         MissingPercent  MissingCount
PopularTimesLivePercent           13.42         48330
AmenityKey                         1.72          6182


In [11]:
# Check Statistical description
fact_live_times['PopularTimesLivePercent'].describe()

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 13, Finished, Available, Finished)

count    311833.000000
mean         35.630597
std          30.625461
min        -100.000000
25%           9.000000
50%          29.000000
75%          55.000000
max         100.000000
Name: PopularTimesLivePercent, dtype: float64

In [12]:
# Replace -100 value in 'popularTimesLivePercent' to NaN 
print("Number of -100 value in PopularTimesLivePercent:", fact_live_times['PopularTimesLivePercent'].eq(-100).sum())
fact_live_times['PopularTimesLivePercent'] = fact_live_times['PopularTimesLivePercent'].replace(-100, np.nan)

# Check latest missing value in 'popularTimesLivePercent'
print("Total Missing value in PopularTimesLivePercent:",fact_live_times['PopularTimesLivePercent'].isna().sum())

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 14, Finished, Available, Finished)

Number of -100 value in PopularTimesLivePercent: 305
Total Missing value in PopularTimesLivePercent: 48635


In [13]:
# Identify missing gap pattern per park 
def get_missing_gap_stats(df, target_col='PopularTimesLivePercent', group_col='ParkName'):
    gap_stats = []

    for park, park_df in df.sort_values('Datetime').groupby(group_col):
        is_missing = park_df[target_col].isna().values
        missing_lengths = []

        count = 0
        for val in is_missing:
            if val:
                count += 1
            elif count > 0:
                missing_lengths.append(count)
                count = 0
        if count > 0:
            missing_lengths.append(count)  # capture trailing gap

        gap_stats.append({
            'ParkName': park,
            'TotalMissing': is_missing.sum(),
            'MissingBlocks': len(missing_lengths),
            'MaxConsecutiveGap': max(missing_lengths) if missing_lengths else 0,
            'AvgGapLength': np.mean(missing_lengths) if missing_lengths else 0
        })

    return pd.DataFrame(gap_stats).sort_values(by='TotalMissing', ascending=False)

# Run it
gap_summary = get_missing_gap_stats(fact_live_times)

display(gap_summary)

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e3efd919-f25b-43c8-a808-478be7f99aa5)

#### **Imputation Techniques Personalized to each Park**

Three methods to fill in missing park occupancy data, depending on the gap patterns:

1. **Group-Based Imputation** is used when data is limited. It fills gaps using average values by hour, weekend, or weekday patterns.

2. **Trend Imputation** is chosen for parks with long, consistent gaps. It uses time and hour trends to estimate missing values.

3. **LightGBM Imputation** is best when there is enough data and the gaps are short but frequent. It uses machine learning to predict values based on features like weather and holidays.

Each park is assessed automatically to select the most suitable method.

In [14]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import numpy as np

def decide_imputation_method(row, min_training_rows=800, max_allowed_gap=1000, min_samples_per_feature=100):
    """
    Enhanced decision logic for imputation method selection
    """
    # Calculate minimum samples needed (rule of thumb: 10-20 samples per feature)
    min_samples_needed = len(['Temp', 'Hour', 'IsWeekend', 'IsHoliday', 'IsRaining', 'IsSnowing', 'HasEvent', 'DayOfWeek']) * min_samples_per_feature
    
    # Estimate available training data
    estimated_available = row["TotalMissing"] * 1.5  # Conservative estimate
    
    # Decision tree
    if estimated_available < min_samples_needed:
        return "enhanced_groupby"
    
    if row["MaxConsecutiveGap"] > max_allowed_gap:
        # For very long gaps, even ML models struggle
        return "temporal_trend" if row["TotalMissing"] > min_training_rows else "enhanced_groupby"
    
    if row["AvgGapLength"] > 100:
        return "temporal_trend"
    
    # High frequency short gaps - perfect for ML
    if row["MissingBlocks"] > 200 and row["AvgGapLength"] < 10:
        return "lightgbm"
    
    # Medium case - try simple ML first
    if estimated_available > min_training_rows:
        return "lightgbm"
    
    return "enhanced_groupby"

def enhanced_groupby_imputation(park_df, features):
    """
    Enhanced groupby that considers multiple factors
    """
    park_df = park_df.copy()
    
    # Try multiple groupby strategies in order of preference
    groupby_strategies = [
        ['Hour', 'IsWeekend'],  # Hour + weekend pattern
        ['Hour', 'DayOfWeek'],  # Hour + day of week
        ['Hour'],               # Just hour
        []                      # Overall mean as last resort
    ]
    
    for strategy in groupby_strategies:
        if len(strategy) > 0:
            # Calculate group means
            group_means = park_df.groupby(strategy)['PopularTimesLivePercent'].transform('mean')
            # Fill missing values where group mean exists
            mask = park_df['PopularTimesLivePercent'].isna() & group_means.notna()
            park_df.loc[mask, 'PopularTimesLivePercent'] = group_means[mask]
        else:
            # Overall mean fallback
            overall_mean = park_df['PopularTimesLivePercent'].mean()
            park_df['PopularTimesLivePercent'].fillna(overall_mean, inplace=True)
            
        # Check if we're done
        if park_df['PopularTimesLivePercent'].isna().sum() == 0:
            break
    
    return park_df

def temporal_trend_imputation(park_df, features):
    """
    Linear trend-based imputation for parks with systematic changes
    """
    park_df = park_df.copy()
    
    # Create a time index (assuming data is sorted by time)
    park_df = park_df.reset_index()
    park_df['time_idx'] = range(len(park_df))
    
    # Get non-missing data
    train_mask = park_df['PopularTimesLivePercent'].notna()
    
    if train_mask.sum() > 50:  # Need minimum data for trend fitting
        # Fit trend + hourly pattern
        train_data = park_df[train_mask].copy()
        
        # Simple linear trend model
        trend_model = LinearRegression()
        X_trend = train_data[['time_idx', 'Hour']].values
        y_trend = train_data['PopularTimesLivePercent'].values
        
        trend_model.fit(X_trend, y_trend)
        
        # Predict missing values
        predict_mask = park_df['PopularTimesLivePercent'].isna()
        if predict_mask.sum() > 0:
            X_predict = park_df.loc[predict_mask, ['time_idx', 'Hour']].values
            predictions = trend_model.predict(X_predict)
            park_df.loc[predict_mask, 'PopularTimesLivePercent'] = predictions
    
    # Fallback to enhanced groupby for any remaining missing values
    if park_df['PopularTimesLivePercent'].isna().sum() > 0:
        park_df = enhanced_groupby_imputation(park_df, features)
    
    return park_df.drop(['time_idx'], axis=1, errors='ignore')

def lightgbm_imputation(park_df, features, min_training_samples=100):
    """
    Enhanced LightGBM imputation with validation
    """
    park_df = park_df.copy()
    
    train_df = park_df[park_df['PopularTimesLivePercent'].notna()]
    predict_df = park_df[park_df['PopularTimesLivePercent'].isna()]
    
    if len(train_df) >= min_training_samples and not predict_df.empty:
        try:
            # Add some feature engineering
            train_features = train_df[features].copy()
            predict_features = predict_df[features].copy()
            
            # Train model with validation
            model = LGBMRegressor(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=6,
                random_state=42,
                verbosity=-1
            )
            
            model.fit(train_features, train_df['PopularTimesLivePercent'])
            
            # Make predictions
            predictions = model.predict(predict_features)
            
            # Clip predictions to reasonable range (0-100 for percentages)
            predictions = np.clip(predictions, 0, 100)
            
            park_df.loc[predict_df.index, 'PopularTimesLivePercent'] = predictions
            
        except Exception as e:
            print(f"LightGBM failed, falling back to enhanced groupby: {e}")
            park_df = enhanced_groupby_imputation(park_df, features)
    else:
        # Fallback to enhanced groupby
        park_df = enhanced_groupby_imputation(park_df, features)
    
    return park_df

# Main imputation function
def impute_park_data(fact_live_times, gap_summary, features):
    """
    Main function to impute missing data with enhanced strategies
    """
    # Apply enhanced decision logic
    gap_summary["RecommendedMethod"] = gap_summary.apply(decide_imputation_method, axis=1)
    
    # Create method dictionary
    method_dict = dict(zip(gap_summary['ParkName'], gap_summary['RecommendedMethod']))
    
    # Prepare container for results
    imputed_list = []
    
    # Process each park
    for park in fact_live_times['ParkName'].unique():
        park_df = fact_live_times[fact_live_times['ParkName'] == park].copy()
        strategy = method_dict.get(park, 'enhanced_groupby')  # fallback
        
        print(f"Processing {park} with strategy: {strategy}")
        
        if strategy == "lightgbm":
            park_df = lightgbm_imputation(park_df, features)
        elif strategy == "temporal_trend":
            park_df = temporal_trend_imputation(park_df, features)
        else:  # enhanced_groupby
            park_df = enhanced_groupby_imputation(park_df, features)
        
        imputed_list.append(park_df)
    
    # Combine results
    df_imputed = pd.concat(imputed_list).sort_index()
    
    # Print summary
    print("\nImputation Summary:")
    method_counts = gap_summary['RecommendedMethod'].value_counts()
    for method, count in method_counts.items():
        print(f"{method}: {count} parks")
    
    return df_imputed, gap_summary

# Usage
features = ['Temp', 'Hour', 'IsWeekend', 'IsHoliday', 'IsRaining', 'IsSnowing', 'HasEvent', 'DayOfWeek']
df_imputed, strategy_summary = impute_park_data(fact_live_times, gap_summary, features)


StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 16, Finished, Available, Finished)

Processing Jonathan Rogers Park with strategy: enhanced_groupby
Processing Rupert Park with strategy: enhanced_groupby
Processing John Hendry (Trout Lake) Park with strategy: enhanced_groupby
Processing Oak Meadows Park with strategy: lightgbm


Processing Guelph Park with strategy: enhanced_groupby
Processing General Brock Park with strategy: enhanced_groupby
Processing CRAB Park at Portside with strategy: enhanced_groupby
Processing Strathcona Park with strategy: enhanced_groupby
Processing Quilchena Park with strategy: lightgbm


Processing Emery Barnes Park with strategy: enhanced_groupby
Processing Memorial South Park with strategy: enhanced_groupby
Processing Clark Park with strategy: enhanced_groupby
Processing Sunset Beach Park with strategy: enhanced_groupby
Processing Queen Elizabeth Park with strategy: temporal_trend


Processing Andy Livingstone Park with strategy: enhanced_groupby
Processing English Bay Beach Park with strategy: enhanced_groupby
Processing Woodland Park with strategy: lightgbm


...

Processing Robson Park with strategy: enhanced_groupby
Processing Oppenheimer Park with strategy: lightgbm


Processing Tatlow Park with strategy: lightgbm


Processing Stanley Park with strategy: lightgbm


Processing Chaldecott Park with strategy: enhanced_groupby
Processing Kaslo Park with strategy: temporal_trend


Processing Coal Harbour Park with strategy: enhanced_groupby
Processing Harbour Green Park with strategy: enhanced_groupby
Processing Maclean Park with strategy: enhanced_groupby
Processing Creekway Park with strategy: enhanced_groupby

Imputation Summary:
enhanced_groupby: 42 parks
lightgbm: 15 parks
temporal_trend: 7 parks


In [15]:
# Drop any remaining missing values and clean up final DataFrame
fact_live_times = (
    df_imputed
    .dropna(subset=['PopularTimesLivePercent'])
    .drop(columns=['index'], errors='ignore')
    .reset_index(drop=True)
)

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 17, Finished, Available, Finished)

In [16]:
# Save as Delta Table to Lakehouse
imputed_fact_live_times = spark.createDataFrame(fact_live_times) #Convert to spark
imputed_fact_live_times.write.format("delta").mode("overwrite").saveAsTable("imputed_fact_live_times")

StatementMeta(, a0ca2bac-edd5-47f3-8a09-eadc8d02d942, 18, Finished, Available, Finished)